# Combine data for NAISE visualization

In [ ]:
import pandas as pd
import json 
import numbers
import re

from datetime import datetime

## Read in the files

In [ ]:
dfProp = pd.read_csv('rawdata/NAISE Proposals All Time V2_new.csv')
dfProp.columns

In [ ]:
dfJA = pd.read_csv('rawdata/NAISE JAs All Time V3_new.csv')
dfJA.columns

In [ ]:
# missing entries in the JA file, but I need to clean up so that the strings will be the same as in JA
dfM = pd.read_csv('rawdata/NAISE unknowns Begum_new.csv')
dfM['department_code'] = dfM['School']
dfM['sub_department'] = dfM['Department/Division']
dfM['last_name'] = dfM['Name'].str.split(',').str[0]
dfM['first_name'] = dfM['Name'].str.split(',').str[1]
# ANL needs to be updated
dfM['sub_department'].unique()

In [ ]:
# append the missing onto the dfJA DataFrame
add_unknowns = pd.DataFrame(columns = dfJA.columns)
add_unknowns['department_code'] = dfM['department_code']
add_unknowns['sub_department'] = dfM['sub_department']
add_unknowns['last_name'] = dfM['last_name']
add_unknowns['first_name'] = dfM['first_name']
dfJA = dfJA.append(add_unknowns)

## Match researchers

There are 25 columns with potential researchers in the proposals file (e.g., 'Researcher 1').  I need to match this with the JA file 'first_name' and 'last_name' column.  I think I will want to take 'department_code', 'sub_department', 'strategic_area',  'affiliation', 'sub_affiliate'.  

I think for the visualization, I will want to create a dataframe with columns that cover the unique values of these, add these columns to the Prop DataFrame, and label them as True/False. 

Since this is a fairly small array, I will just match by a brute force loop.  

In [ ]:
# I will create a dict that will hold all of these columns then fill them below
department_code = dfJA['department_code'].dropna().unique()
sub_department = dfJA['sub_department'].dropna().unique()
strategic_area = dfJA['strategic_area'].dropna().unique()
affiliation = dfJA['affiliation'].dropna().unique()
sub_affiliate = dfJA['sub_affiliate'].dropna().unique()

print(department_code)
print(sub_department)
print(strategic_area)
print(affiliation)
print(sub_affiliate)

In [ ]:
# we will use the department_code and sub_department, but should I clean up the sub_department to only have the acronym?
sub_department_acronym = []
for s in sub_department:
    x = s.split()
    sub_department_acronym.append(x[-1].replace('(','').replace(')',''))
#sub_department_acronym

In [ ]:
# check for new sub_departments in unknowns file
for s in dfM['sub_department'].unique():
    if (s not in sub_department):
        print(s)

In [ ]:
N = len(dfProp)
add_columns = {}
add_columns['department_code'] = {}
for d in department_code:
    add_columns['department_code'][d] = [0]*N
    
add_columns['sub_department'] = {}
for d in sub_department:
    add_columns['sub_department'][d] = [0]*N
    
# add_columns['strategic_area'] = {}
# for d in strategic_area:
#     add_columns['strategic_area'][d] = [0]*N
    
# add_columns['affiliation'] = {}
# for d in affiliation:
#     add_columns['affiliation'][d] = [0]*N
    
# add_columns['sub_affiliate'] = {}   
# for d in sub_affiliate:
#     add_columns['sub_affiliate'][d] = [0]*N
    
#add_columns

## I'm going to try the Hierarchical Edge Bundling example

following this example : https://bl.ocks.org/mbostock/1044242

Along the exterior will be people's names.  Eventually I will remove those names and put in the sub_department labels.  I will also work out how to re-color the lines and maybe also change their thickness.  

I will format the data in the same way as the example, and give the "name" key as the PI.  I'm not sure how exactly it will work if I have the same PI multiple times, but I will at least print it out here to see how often that happens.  I may need to create additional dummy PIs to avoid duplicates.

Also there are some (many?) grants that only have the PI.  I think I will need to include the PI in the researchers for those so that there is at least a line looping back.  This would also require a dummy PI.



In [ ]:
def match_researcher(researcher):
    dept = 'none'
    sub = 'none'
        
    xx = researcher.split(',')
    last_name = xx[0].strip()
    first_name = xx[1].split()[0].strip()
    # use first name
    usedf = dfJA.loc[(dfJA['first_name'].str.contains(first_name, na = False)) &
                 (dfJA['last_name'].str.contains(last_name, na = False))]
    
    if (len(usedf) > 1):
        # use middle initial
        yy = xx[1].split()
        if (len(yy) > 1):
            middle = ' ' + yy[1].strip().replace('.','')
            usedf = dfJA.loc[(dfJA['first_name'].str.contains(first_name, na = False)) &
                 (dfJA['first_name'].str.contains(middle, na = False)) &
                 (dfJA['last_name'].str.contains(last_name, na = False))]
                
        if (len(usedf) != 1):
            print(f'multiple entries, name:{researcher}, first:{first_name}, last:{last_name}, len:{len(usedf)}')

        # get the info

    if (len(usedf) > 0):
        rr = usedf.iloc[0]

        # check to see if we have the department_code and sub_department
        if (not pd.isnull(rr['department_code'])):
            dept = rr['department_code'].strip()#.replace(' ','')
        if (not pd.isnull(rr['sub_department'])):
            sub = rr['sub_department'].strip()#.replace(' ','')

    if (len(usedf) == 0):
        print('missing', researcher)
                                
    #out = 'NAISE.' + dept + '.' + sub + '.' + first_name + last_name
    out = dept + '.' + sub + '.' + first_name + last_name
    
    #return re.sub(r'[^a-zA-Z0-9.]', '', out)
    return out

def mapped(x):
    if isinstance(x,numbers.Number):
        return x
    for tpe in (int, float):
        try:
            return tpe(x)
        except ValueError:
            continue
    return 0

def addNum(name, nameList):
    matching = sorted([s for s in nameList if name in s])
    num = 0
    if (len(matching) > 0):
        num = mapped(matching[-1][-3:]) + 1
    return name + str(num).zfill(3)
    
        
# Note: this might be cleaner as a dict or DataFrame, but this works well enough for now
out = [] #this will be a list of dicts
researchers = [] #to keep track of the PI and researchers, so that there aren't any duplicates
researchersYears = [] #to hold all year for the PIs and reseachers
researchersDollars = [] #to hold all dollars for the PIs and reseachers
researchersFunded = [] #to hold all funded flags for the PIs and reseachers
researchersGnumber = [] #to hold all grant numbers for the PIs and reseachers
names = [] #to hold all the names that get dicts (each PI and researcher needs a dict)

# should I include the PI here?  For some grants, there is only a PI w/o any researchers
# for those if I don't include the PI, I don't think anything would be drawn.
# though still nothing may be drawn because the line won't go anywhere.  
# this could be another reason why I might need to create additional PIs
# I will not include the PI here, but I will create a dummy PI to add as a researcher so there is a loop.
researcher_cols = ['Researcher 1', 'Researcher 2',
       'Researcher 3', 'Researcher 4', 'Researcher 5', 'Researcher 6',
       'Researcher 7', 'Researcher 8', 'Researcher 9', 'Researcher 10',
       'Researcher 11', 'Researcher 12', 'Researcher 13', 'Researcher 14',
       'Researcher 15', 'Researcher 16', 'Researcher 17', 'Researcher 18',
       'Researcher 19', 'Researcher 20', 'Researcher 21', 'Researcher 22',
       'Researcher 23', 'Researcher 24', 'Researcher 25']

missing = []
bad = []
dfJA['matched'] = [False]*len(dfJA)

for i,row in dfProp.iterrows():
    # get the year
    year = int(row['Proposal Submit Date'].split('/')[-1])# + 2000
    if (year == 0 and not pd.isnull(row['Proposed Project Begin'])):
        year = int(row['Proposed Project Begin'].split('/')[-1])# + 2000
    if (year == 0):
        print('bad year', year, row['PI'], row['Proposal Submit Date'], row['Institution Number'])

    # get the dollar amount (min at 1)
    x = re.sub('[\$,-]', '', row['Total Proposed to Sponsor'])
    dollars = max(1., pd.to_numeric(x, errors='coerce').astype(float))
    
    # get the grant number
    gnumber = row['Institution Number']
    
    # check if the grant was funded
    funded = 'submitted'
    if (row['Proposal Status'] == 'Awarded QA Check Complete' or row['Proposal Status'] == 'Funded On Main'):
        funded = 'funded'
        # also check if the grant is active, given the dates
        date0 = datetime.strptime(row['Proposed Project Begin'], '%m/%d/%Y')
        date1 = datetime.strptime(row['Proposed Project End'], '%m/%d/%Y')
        if (datetime.today() > date0 and datetime.today() < date1):
            funded = 'active'
            #print(funded, date0, date1, gnumber, dollars)
                                  
        

    
    # get the PI 
    PI = addNum(match_researcher(row['PI']), researchers)
    researchers.append(PI)
    researchersYears.append(year)
    researchersDollars.append(dollars)
    researchersFunded.append(funded)
    researchersGnumber.append(gnumber)

    row_dict = {'name':PI, 'dollars':dollars, 'year':year, 'funded':funded, 'grantNumber':gnumber, 'researchers':[]}
               
    for rc in researcher_cols:
        if (not pd.isnull(row[rc])):
            researcher = addNum(match_researcher(row[rc]), researchers)
            if (researcher != PI):
                row_dict['researchers'].append(researcher)
                researchers.append(researcher)
                researchersYears.append(year)
                researchersDollars.append(dollars)
                researchersFunded.append(funded)
                researchersGnumber.append(gnumber)

    #check to see if any researchers were there
    if (len(row_dict['researchers']) == 0):
        # create a dummy PI and add that to the researchers
        PI = addNum(match_researcher(row['PI']), researchers)
        row_dict['researchers'].append(PI)
        researchers.append(PI)
        researchersYears.append(year)
        researchersDollars.append(dollars)
        researchersFunded.append(funded)
        researchersGnumber.append(gnumber)

    out.append(row_dict)
    names.append(row_dict['name'])
    
# append empty entries for all the rest of the researchers so they can appear as nodes
researchersUnique = set(list(researchers))
for r in researchersUnique:
    if (r not in names):
        indices = [i for i, x in enumerate(researchers) if x == r]
        years = [researchersYears[i] for i in indices]
        dollars = [researchersDollars[i] for i in indices]
        funded = [researchersFunded[i] for i in indices]
        gnumber = [researchersGnumber[i] for i in indices]
        if (len(indices) > 1):
            print(r, indices, years, dollars, funded)
        out.append({'name':r, 'dollars':dollars, 'year':years[0], 'funded':funded[0], 'grantNumber':gnumber[0], 'researchers':[]})

# sort so that similar departments are next to each other
# I want to put NU-NAISE at the start/end of the Northwestern side, so I will modify the sort command
#out.sort(key = lambda item: item['name'] if (not item['name'].startswith('NU-NAISE')) else 'x'+item['name'])
out.sort(key = lambda item: item['name'] 
         if (not item['name'].startswith('NU-FEIN') and not 'BarrySmith' in item['name']) 
         else item['name'].replace('NU-FEIN', 'NU-MFEIN').replace('BarrySmith', 'PBarrySmith')
        )
#print(out)

In [ ]:
with open('NAISE_bundling.json', 'w') as outfile:
    json.dump(out, outfile)

In [ ]:
# check depts
dept = []
for d in out:
    dept.append(d['name'].rsplit('.', 1)[0])
dept = sorted(list(set(dept)))
dept

In [ ]:
yrs = [x for x in researchersYears if x > 0]
min(yrs)

In [ ]:
max(researchersDollars)

In [ ]:
# check with grants have any researchers with NU-NAISE as the base
check = []
for d in out:
    show = False
    checkd = {'grantNumber': d['grantNumber'], 'NAISEResearchers':[]}
    if ('NU-NAISE' in d['name']):
        show = True
        checkd['NAISEResearchers'].append(d['name'])
    for r in d['researchers']:
        if ('NU-NAISE' in r):
            show = True
            checkd['NAISEResearchers'].append(r)

    if (show):
        check.append(checkd)
        
for c in check:
    print(c)


In [ ]:
for d in out:
    #if ('none.' in d['name']):
    #if ('ANL-PSE.' in d['name']):
    #if ('KatrinHeitmann' in d['name']):
    if ('ANL-CLS.Mathematics and Computer Science (MCS)' in d['name']):
        print(d['name'])

In [ ]:
for d in out:
    if (d['grantNumber'] == 'SP0025446'):
        print(d['name'])

## Create a bar chart showing number of people from the different departments on the grants

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

%matplotlib inline

In [ ]:
#gather all the unique people
uniquePeople = []
for d in out:
    dd = d['name'][:-3]
    if (dd not in uniquePeople):
        uniquePeople.append(dd)
        print(dd)


In [ ]:
dept = []
for d in uniquePeople:
    dd = d.rsplit('.', 1)[0]
    if (dd not in dept):
        dept.append(dd)

deptN = [0]*len(dept)
for o in uniquePeople:
    d = o.rsplit('.', 1)[0]
    i = dept.index(d)
    if (i >= 0):
        deptN[i] += 1
    else:
        print('missing dept', d)

deptT = []
for d in dept:
    deptT.append(d.replace('.',' - '))
    
deptColors = {
    'ANL-CLS'   : '#0045cf',  
    'ANL-EGS'   : '#3366CC', 
    'ANL-PSC'   : '#6592f0',  
    'ANL-PSE'   : '#98b6f5', 
    'NU-MCC'    : '#4E2A84', 
    'NU-FEIN'   : '#684c96', 
    'NU-Medill' : '#836EAA', 
    'NU-WCAS'   : '#a495c3' 
}

deptC = []
deptCode = []
for d in dept:
    dd = d[0:d.find('.')]
    deptCode.append(dd)
    deptC.append(deptColors[dd])
    
lns = []
for i,d in enumerate(deptCode):
    if (i > 0):
        if (deptCode[i - 1] != d):
            lns.append(i)

In [ ]:
font = {'size' : 20}
matplotlib.rc('font', **font) 

f,ax = plt.subplots(figsize=(10,20))

ind = [i for i in range(len(dept))]  # the x locations for the bars
width = 0.7 # the width of the bars
rects = ax.barh(ind, deptN, width, zorder=2)

# add some text for labels, title and axes ticks
ax.set_yticks(ind)
ax.set_yticklabels(deptT)

lbl = ax.get_yticklabels()
for i,l in enumerate(lbl):
    l.set_color(deptC[i]) 

for l in lns:
    ax.annotate('',
        xy = (0, l - 0.5), xycoords='data',
        xytext = (-700, 1), textcoords='offset points',
        arrowprops = dict(facecolor = 'black', arrowstyle = "-"),
        horizontalalignment = 'right', verticalalignment = 'bottom')

#remove all the axes, ticks and lower x label
#aoff = ['right', 'left', 'top', 'bottom']
aoff = ['right', 'top', 'bottom']
for x in aoff:
    ax.spines[x].set_visible(False)
ax.tick_params(length  =0)
_ = ax.set_xticklabels([' ']*len(dept))



#I grabbed this from here : https://matplotlib.org/examples/api/barchart_demo.html
#and tweaked it slightly 
highlight = [4]
for i, r in enumerate(rects):
    h = r.get_height()
    w = r.get_width()
    y = r.get_y()
    c = 'white'
    if (w >= 10):
        x = w - 0.6
    else:
        x = w - 0.3

#     elif (w > 2):
#         x = w - 0.3
#     else:
#         x = w + 0.3
#         c = 'black'
    r.set_color(deptC[i])
    ax.text(x, y ,'%i' % w, ha='center', va='bottom', zorder = 3, color=c) 
    
f.savefig('NAISE_people_bar.pdf', format = 'pdf', bbox_inches = 'tight')